### LIBRERÍAS

In [1]:
import pandas as pd
import mysql.connector

### PROCESAMIENTO

In [2]:
df_ratin_final = pd.read_csv("Data/rating_final.csv")
df_accepts = pd.read_csv("Data/chefmozaccepts.csv")
df_cousine = pd.read_csv("Data/chefmozcuisine.csv")
df_houors4 = pd.read_csv("Data/chefmozhours4.csv")
df_geoplaces = pd.read_csv("Data/geoplaces2.csv")
df_parking = pd.read_csv("Data/chefmozparking.csv")

In [4]:
lista_id = df_geoplaces["placeID"].unique()

df_accepts = df_accepts[df_accepts['placeID'].isin(lista_id)]
df_ratin_final = df_ratin_final[df_ratin_final['placeID'].isin(lista_id)]
df_cousine = df_cousine[df_cousine['placeID'].isin(lista_id)]
df_houors4 = df_houors4[df_houors4['placeID'].isin(lista_id)]
df_parking = df_parking[df_parking['placeID'].isin(lista_id)]

df_ratin_final.drop("userID", axis=1, inplace=True)

### FUNCIONES SQL (faltaría la contraseña de cada uno)

In [6]:
# ESTABLECER CONEXIÓN CON LA BASE DE DATOS DE MYSQL
# PARA GUARDAR LA CONEXIÓN Y EL CURSOR, HAY QUE IGUALAR LA FUNCIÓN A DOS VARIABLES
def conexion():
    db = mysql.connector.connect(host     = "localhost", # donde se aloja la base de datos
                             user     = "root", # poned vuestro usuario
                             password = "", # poned vuestra contraseña
                             database = None)
    cursor = db.cursor()
    return db, cursor

# CERRAR LA CONEXIÓN
def cerrar_conexion(cursor, db):
    cursor.fetchall() # se utiliza para recuperar todas las filas del conjunto de resultados después de ejecutar una consulta en una base de datos. Esto es útil cuando deseas trabajar con los resultados de una consulta en tu programa Python
    cursor.close()
    db.close()

# CREAR UNA BASE DE DATOS DANDOLE EL NOMBRE DE LA BASE DE DATOS
def crear_database(database):
    db, cursor = conexion()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
    cerrar_conexion(cursor, db)

# CREAR UNA TABLA EN UNA BASE DE DATOS.
# PASANDOLE LA BASE DE DATOS DONDE QUEREMOS CREARLA Y LA QUERY DE LA TABLA QUE QUEREMOS CREAR
def crear_tablas(query, database):
    db, cursor = conexion()
    query_database = f"USE {database};"
    cursor.execute(query_database)
    cursor.execute(query)
    cerrar_conexion(cursor, db)

# EJECUTAR QUERYS DE CONSULTAS:
def ejecutar_consulta(database, query):
    db, cursor = conexion()
    cursor.execute(f"USE {database};")

    # Ejecutar la consulta
    cursor.execute(query)
    columnas = cursor.column_names
    # Obtener los resultados, si es una consulta SELECT
    resultados = cursor.fetchall()
    # Hacer commit para guardar los cambios (en caso de INSERT, UPDATE, DELETE, etc.)
    db.commit()
    # Cerrar la conexión
    cerrar_conexion(cursor, db)
    return resultados, columnas

def insertar_datos_porfila(df, database, tabla):
    db, cursor = conexion()
    cursor.execute(f"USE {database};")

    # Iterar sobre las filas del DataFrame e insertar cada fila en la tabla
    for _, fila in df.iterrows(): #Este método de pandas retorna un generador que produce pares (índice, fila) para cada fila del DataFrame df. 
                                  #el guion bajo _ se utiliza para descartar el índice, ya que no se está utilizando dentro del bucle.
                                  #Aquí el isinstance verifica que le valor es un str, sino lo convierte a str ya que la query tiene que ser str
        valores = ", ".join([f"'{valor}'" if isinstance(valor, str) else str(valor) for valor in fila])
        query = f"INSERT INTO {tabla} VALUES ({valores});"
        cursor.execute(query)

    # Hacer commit para guardar los cambios
    db.commit()
    cerrar_conexion(cursor, db)

### CREACIÓN DATABASE

In [ ]:
crear_database("recomendador")

### QUERYS

In [8]:
query_geoplaces = """CREATE TABLE IF NOT EXISTS geoplaces (
                    placeID INT UNSIGNED NOT NULL,
                    latitude FLOAT, 
                    longitude FLOAT, 
                    the_geom_meter VARCHAR(255), 
                    name VARCHAR(255), 
                    address VARCHAR(255),
                    city VARCHAR(255), 
                    state VARCHAR(255), 
                    country VARCHAR(255), 
                    fax VARCHAR(255), 
                    zip VARCHAR(255), 
                    alcohol VARCHAR(255), 
                    smoking_area VARCHAR(255),
                    dress_code VARCHAR(255), 
                    accessibility VARCHAR(255), 
                    price VARCHAR(255), 
                    url VARCHAR(255), 
                    Rambience VARCHAR(255), 
                    franchise VARCHAR(255),
                    area VARCHAR(255), 
                    other_services VARCHAR(255),
                    PRIMARY KEY (placeID)
                    );"""

query_payments = """CREATE TABLE IF NOT EXISTS payments (
                    placeID INT UNSIGNED NOT NULL,
                    Rpayment VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_cocina = """CREATE TABLE IF NOT EXISTS cocina (
                    placeID INT UNSIGNED NOT NULL,
                    Rcuisine VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_horario = """CREATE TABLE IF NOT EXISTS horario (
                    placeID INT UNSIGNED NOT NULL,
                    hours VARCHAR(255),
                    days VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_parking = """CREATE TABLE IF NOT EXISTS parking (
                    placeID INT UNSIGNED NOT NULL,
                    parking_lot VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_rating = """CREATE TABLE IF NOT EXISTS rating (
                    placeID INT UNSIGNED NOT NULL,
                    rating INT UNSIGNED NOT NULL,
                    food_rating INT UNSIGNED NOT NULL,
                    service_rating INT UNSIGNED NOT NULL,
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

### CREACIÓN TABLAS

In [11]:
crear_tablas(query_geoplaces, "recomendador")
crear_tablas(query_payments, "recomendador")
crear_tablas(query_cocina, "recomendador")
crear_tablas(query_horario, "recomendador")
crear_tablas(query_parking, "recomendador")
crear_tablas(query_rating, "recomendador")

### INSERCCIÓN DATOS

In [14]:
insertar_datos_porfila(df_geoplaces, "recomendador", "geoplaces")
insertar_datos_porfila(df_accepts, "recomendador", "payments")
insertar_datos_porfila(df_cousine, "recomendador", "cocina")
insertar_datos_porfila(df_houors4, "recomendador", "horario")
insertar_datos_porfila(df_parking, "recomendador", "parking")
insertar_datos_porfila(df_ratin_final, "recomendador", "rating")

### EXTRACCIÓN DATOS

In [2]:
query_select = """SELECT * FROM geoplaces
          JOIN payments ON geoplaces.placeID=payments.placeID
          JOIN cocina ON geoplaces.placeID=cocina.placeID
          JOIN horario ON geoplaces.placeID=horario.placeID
          JOIN parking ON geoplaces.placeID=parking.placeID
          JOIN rating ON geoplaces.placeID = rating.placeID
          ORDER BY geoplaces.placeID;
        """

In [7]:
resultado, columnas = ejecutar_consulta("recomendador", query_select)

In [8]:
df = pd.DataFrame(resultado, columns = columnas)

In [9]:
# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

In [10]:
df

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,area,other_services,Rpayment,Rcuisine,hours,days,parking_lot,rating,food_rating,service_rating
0,132560,23.7523,-99.1669,0101000020957F0000FC60BDA8E88157C1B2C357D6DA4E...,puesto de gorditas,frente al tecnologico,victoria,tamaulipas,mexico,?,...,open,none,cash,Regional,08:00-12:00;,Mon;Tue;Wed;Thu;Fri;,public,1,0,0
1,132560,23.7523,-99.1669,0101000020957F0000FC60BDA8E88157C1B2C357D6DA4E...,puesto de gorditas,frente al tecnologico,victoria,tamaulipas,mexico,?,...,open,none,cash,Regional,08:00-12:00;,Mon;Tue;Wed;Thu;Fri;,public,0,0,0
2,132560,23.7523,-99.1669,0101000020957F0000FC60BDA8E88157C1B2C357D6DA4E...,puesto de gorditas,frente al tecnologico,victoria,tamaulipas,mexico,?,...,open,none,cash,Regional,08:00-12:00;,Mon;Tue;Wed;Thu;Fri;,public,0,2,0
3,132560,23.7523,-99.1669,0101000020957F0000FC60BDA8E88157C1B2C357D6DA4E...,puesto de gorditas,frente al tecnologico,victoria,tamaulipas,mexico,?,...,open,none,cash,Regional,08:00-12:00;,Mon;Tue;Wed;Thu;Fri;,public,1,2,1
4,132560,23.7523,-99.1669,0101000020957F0000FC60BDA8E88157C1B2C357D6DA4E...,puesto de gorditas,frente al tecnologico,victoria,tamaulipas,mexico,?,...,open,none,cash,Regional,00:00-00:00;,Sat;,public,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6271,135109,18.9218,-99.2354,0101000020957F0000A6BF695F136F5AC1DADF87B20556...,Paniroles,?,?,?,?,?,...,closed,Internet,cash,Italian,08:00-21:00;,Sat;,none,1,2,1
6272,135109,18.9218,-99.2354,0101000020957F0000A6BF695F136F5AC1DADF87B20556...,Paniroles,?,?,?,?,?,...,closed,Internet,cash,Italian,08:00-21:00;,Sun;,none,0,0,0
6273,135109,18.9218,-99.2354,0101000020957F0000A6BF695F136F5AC1DADF87B20556...,Paniroles,?,?,?,?,?,...,closed,Internet,cash,Italian,08:00-21:00;,Sun;,none,2,2,1
6274,135109,18.9218,-99.2354,0101000020957F0000A6BF695F136F5AC1DADF87B20556...,Paniroles,?,?,?,?,?,...,closed,Internet,cash,Italian,08:00-21:00;,Sun;,none,1,1,1


### PROCESAMIENTO

In [11]:
# Crear columnas para cada día y rellenarlas con los horarios correspondientes

df['Entre Semana'] = df.apply(lambda row: row['hours'] if row['days'] == 'Mon;Tue;Wed;Thu;Fri;' else None, axis=1)
df['Sabado'] = df.apply(lambda row: row['hours'] if row['days'] == 'Sat;' else None, axis=1)
df['Domingo'] = df.apply(lambda row: row['hours'] if row['days'] == 'Sun;' else None, axis=1)

df = df.drop(['hours', 'days'], axis=1)

# Haciendo dummies manual a la vez que pivotear para tener los datos en una sola fila
for restaurante in df["name"].unique():

    df.loc[df["name"] == restaurante, "Entre Semana"] = df.loc[df["name"] == restaurante, "Entre Semana"].dropna().values[0]
    df.loc[df["name"] == restaurante, "Sabado"] = df.loc[df["name"] == restaurante, "Sabado"].dropna().values[0]
    df.loc[df["name"] == restaurante, "Domingo"] = df.loc[df["name"] == restaurante, "Domingo"].dropna().values[0]

print(df.duplicated().sum())

df.drop_duplicates(inplace=True)


5068


In [12]:
# Procesando ratings

for restaurante in df["name"].unique():
    numerador = (df[df["name"] == restaurante]["rating"].sum())
    denominador = (df[df["name"] == restaurante].shape[0])
    
    # Calcular el rating_ana y asignar los valores a las filas correspondientes
    df.loc[df["name"] == restaurante, "rating"] = numerador / denominador

for restaurante in df["name"].unique():
    numerador = (df[df["name"] == restaurante]["food_rating"].sum())
    denominador = (df[df["name"] == restaurante].shape[0])
    
    # Calcular el rating_ana y asignar los valores a las filas correspondientes
    df.loc[df["name"] == restaurante, "food_rating"] = numerador / denominador

for restaurante in df["name"].unique():
    numerador = (df[df["name"] == restaurante]["service_rating"].sum())
    denominador = (df[df["name"] == restaurante].shape[0])
    
    # Calcular el rating_ana y asignar los valores a las filas correspondientes
    df.loc[df["name"] == restaurante, "service_rating"] = numerador / denominador

print(df.duplicated().sum())

df.drop_duplicates(inplace=True)

993


In [13]:
df["name"].value_counts()

name
La Fontana Pizza Restaurante and Cafe    12
Restaurante la Cantina                    8
Cafeteria y Restaurant El Pacifico        6
Mcdonalds Parque Tangamanga               6
Restaurante Bar El Gallinero              6
                                         ..
cafe punta del cielo                      1
el pueblito                               1
rockabilly                                1
VIPS                                      1
Paniroles                                 1
Name: count, Length: 84, dtype: int64

In [14]:
# Dummies para tipo de cocina
df_pagos = pd.get_dummies(df['Rpayment'])

df = pd.concat([df, df_pagos], axis=1)

# Pivotando
for categoria in df["Rpayment"].unique():
    for restaurante in df["name"].unique():
        if df.loc[df["name"] == restaurante, categoria].nunique() == 2:
            df.loc[df["name"] == restaurante, categoria] = 1
        else:
            df.loc[df["name"] == restaurante, categoria] = 0

df.drop("Rpayment", axis=1, inplace=True)

print(df.duplicated().sum())

df.drop_duplicates(inplace=True)

116


In [15]:
df["name"].value_counts()

name
La Fontana Pizza Restaurante and Cafe    3
Restaurante Bar El Gallinero             2
Preambulo Wifi Zone Cafe                 2
Cafeteria y Restaurant El Pacifico       2
Luna Cafe                                2
                                        ..
Hamburguesas Valle Dorado                1
Unicols Pizza                            1
Sirlone                                  1
el lechon potosino                       1
Paniroles                                1
Name: count, Length: 84, dtype: int64

In [16]:
# Dummies para tipo de cocina
df_cocina = pd.get_dummies(df['Rcuisine'])

df = pd.concat([df, df_cocina], axis=1)

# Pivotando
for categoria in df["Rcuisine"].unique():
    for restaurante in df["name"].unique():
        if df.loc[df["name"] == restaurante, categoria].nunique() == 2:
            df.loc[df["name"] == restaurante, categoria] = 1
        else:
            df.loc[df["name"] == restaurante, categoria] = 0

df.drop("Rcuisine", axis=1, inplace=True)

print(df.duplicated().sum())

df.drop_duplicates(inplace=True)

14


In [17]:
# Seleccionar las filas para el restaurante específico
restaurante_seleccionado = "Gorditas Dona Tota"
df_restaurante = df[df["name"] == restaurante_seleccionado]

# Encontrar las columnas con más de un valor único
columnas_con_diferencias = df_restaurante.columns[df_restaurante.nunique() > 1]

In [18]:
# Mostrar los valores únicos para las columnas con diferencias
for columna in columnas_con_diferencias:
    valores_unicos = df_restaurante[columna].unique()
    print(f'Columna: {columna}, Valores únicos: {valores_unicos}')

Columna: placeID, Valores únicos: [132584 132706]
Columna: latitude, Valores únicos: [23.7524 23.7292]
Columna: longitude, Valores únicos: [-99.1653 -99.1324]
Columna: the_geom_meter, Valores únicos: ['0101000020957F000048C65BA7EF8157C177F2344D664E4941'
 '0101000020957F00000F14BF6B2C8657C1963CCB8E5C464941']
Columna: address, Valores únicos: ['?' 'Zaragoza entre Francisco Zarco y Lopez Velarde']
Columna: city, Valores únicos: ['?' 'Cd. Victoria']
Columna: state, Valores únicos: ['?' 'Tamaulipas']
Columna: country, Valores únicos: ['?' 'Mexico']
Columna: accessibility, Valores únicos: ['completely' 'no_accessibility']
Columna: parking_lot, Valores únicos: ['yes' 'public']


In [19]:
df["name"].value_counts()

name
Gorditas Dona Tota            2
puesto de gorditas            1
Michiko Restaurant Japones    1
Restaurante Bar Fu-hao        1
Restaurante El Reyecito       1
                             ..
Unicols Pizza                 1
Sirlone                       1
KFC                           1
el lechon potosino            1
Paniroles                     1
Name: count, Length: 84, dtype: int64

In [20]:
# Filtrando por interés
df = df[df["name"] != "Gorditas Dona Tota"]

In [21]:
df.reset_index(drop=True, inplace=True)

In [22]:
df.drop(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'address', 'city', 'state', 'country', 'fax', 'zip', 'franchise', 'url', 'other_services'], axis=1, inplace=True)

In [152]:
df.to_csv("Data/Restaurantes_mexico.csv")

# RECOMENDADOR

In [23]:
# Aplicar one-hot encoding a varias columnas
columnas_a_codificar = ['alcohol', 'smoking_area', 'dress_code', 'accessibility', 'price', 'Rambience', 'area', 'parking_lot']

df_dummies = pd.get_dummies(df, columns=columnas_a_codificar)

df_dummies.drop(["smoking_area_none", "rating",	"food_rating",	"service_rating",	"Entre Semana",	"Sabado","Domingo"], axis=1, inplace=True)

In [24]:
# Identificar columnas con tipo de dato booleano
columnas_booleanas = df_dummies.select_dtypes(include='bool').columns

# Cambiar el tipo de dato de booleano a entero solo para las columnas booleanas
df_dummies[columnas_booleanas] = df_dummies[columnas_booleanas].astype(int)

### RESTAURANTES QUE PUEDE PUNTUAR EL USUARIO

In [25]:
df_dummies.name.to_list()

['puesto de gorditas',
 'Cafe Chaires',
 'tacos de barbacoa enfrente del Tec',
 'Hamburguesas La perica',
 'Pollo_Frito_Buenos_Aires',
 'carnitas_mata',
 'la perica hamburguesa',
 'little pizza Emilio Portes Gil',
 'tacos de la estacion',
 'tortas hawai',
 'Gordas de morales',
 'Taqueria EL amigo ',
 'Little Cesarz',
 'Carreton de Flautas y Migadas',
 'Cabana Huasteca',
 'La Estrella de Dimas',
 'Mikasa',
 'Restaurant Familiar El Chino',
 'Mariscos Tia Licha',
 'El cotorreo',
 'puesto de tacos',
 'Gorditas Doa Gloria',
 'Cenaduria El Rincón de Tlaquepaque',
 'el lechon potosino ',
 'KFC',
 'Sirlone',
 'Unicols Pizza',
 'Hamburguesas Valle Dorado',
 'Carls Jr',
 'La Posada del Virrey',
 'Chaires',
 'Dominos Pizza',
 'Tortas y hamburguesas el gordo',
 'Pizzeria Julios',
 'shi ro ie',
 'Hamburguesas saul',
 'crudalia',
 'cafe punta del cielo',
 'el pueblito',
 'rockabilly ',
 'VIPS',
 'la parroquia',
 'emilianos',
 'tacos los volcanes',
 'Restaurant and Bar and Clothesline Carlos N Charli

### VALORACIÓN DEL USUARIO

In [26]:
datos = [
            ['Mcdonalds Parque Tangamanga', 6],
            ['Cafeteria cenidet', 3],
            ['vips', 9],
            ['El Rinc�n de San Francisco', 4],
            ['Paniroles', 6]
        ]

df_usuario = pd.DataFrame(data = datos, columns = ["name", "Puntuacion"])

### ALGORITMO RECOMENDADOR

In [27]:
# Concatenar los DataFrames basándose en la columna común
df_usuario = pd.merge(df_dummies, df_usuario, on='name', how='left')

In [28]:
df_pesos = df_usuario[df_usuario["Puntuacion"] >0].copy()

In [29]:
# Asignandole valor a las caracteristicas de cada restaurante segun la puntuacion que dió el usuario
for columna in df_pesos.columns:
    if columna != "name" and columna != "Puntuacion":
        df_pesos[columna] = df_pesos[columna]*df_pesos["Puntuacion"]

In [30]:
# Calculando el peso de cada característica
usuario_pesos = df_pesos.drop(["name", "Puntuacion"], axis=1).sum()

usuario_pesos = usuario_pesos/usuario_pesos.sum()

In [31]:
# Creando df de restaurantes que no visitó el usuario
df_restaurantes = df_usuario[~(df_usuario["Puntuacion"] > 0)].copy()

In [32]:
# Asignandole el peso correspondientes a cada característica según el usuario de los demás restaurantes
weighted_movies_matrix = list()

for punto, restaurant in zip(usuario_pesos.values, df_restaurantes.iloc[:, 1:-1].values):
    weighted_movies_matrix.append(punto*restaurant)

weighted_movies_matrix = pd.DataFrame(data = weighted_movies_matrix, columns = df_restaurantes.drop(["name", "Puntuacion"], axis=1).columns)

In [33]:
# Creando df de donde saldrá las recomendaciones
df_recomendador = pd.concat([df_restaurantes, weighted_movies_matrix.sum(axis = 1)], axis = 1)

df_recomendador.rename(columns= {0 : "Recomendacion"}, inplace=True)
# Ordenando los restaurantes de mayor peso a menor
df_recomendador.sort_values("Recomendacion", ascending = False, inplace=True)

In [34]:
# Filtrado de recomendaciones para el usuario
resultado = df_recomendador[df_recomendador["Recomendacion"] > 0]["name"].to_list()

In [35]:
# De mejor recomendación a peor:
resultado

['tacos los volcanes',
 'Restaurant and Bar and Clothesline Carlos N Charlies',
 'cafe punta del cielo',
 'Cafeteria y Restaurant El Pacifico',
 'Dominos Pizza',
 'Subway',
 'tacos de barbacoa enfrente del Tec',
 'Hamburguesas La perica',
 'carnitas_mata',
 'rockabilly ',
 'Carls Jr',
 'Restaurant Orizatlan',
 'El Oceano Dorado',
 'La Posada del Virrey',
 'Chaires',
 'el pueblito',
 'Little Cesarz',
 'Mariscos Tia Licha',
 'El Rincon de San Francisco',
 'emilianos',
 'La Virreina',
 'VIPS',
 'Tortas y hamburguesas el gordo']